In [1]:
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as la
## Construct LSTM using Tensorflow + Keras
# Import Libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras import optimizers


######################################################################
######################################################################
######################################################################
## CSP File from: https://github.com/spolsley/common-spatial-patterns

# CSP takes any number of arguments, but each argument must be a collection of trials associated with a task
# That is, for N tasks, N arrays are passed to CSP each with dimensionality (# of trials of task N) x (feature vector)
# Trials may be of any dimension, provided that each trial for each task has the same dimensionality,
# otherwise there can be no spatial filtering since the trials cannot be compared
def CSP(*tasks):
	if len(tasks) < 2:
		print("Must have at least 2 tasks for filtering.")
		return (None,) * len(tasks)
	else:
		filters = ()
		# CSP algorithm
		# For each task x, find the mean variances Rx and not_Rx, which will be used to compute spatial filter SFx
		iterator = range(0,len(tasks))
		for x in iterator:
			# Find Rx
			Rx = covarianceMatrix(tasks[x][0])
			for t in range(1,len(tasks[x])):
				Rx += covarianceMatrix(tasks[x][t])
			Rx = Rx / len(tasks[x])

			# Find not_Rx
			count = 0
			not_Rx = Rx * 0
			for not_x in [element for element in iterator if element != x]:
				for t in range(0,len(tasks[not_x])):
					not_Rx += covarianceMatrix(tasks[not_x][t])
					count += 1
			not_Rx = not_Rx / count

			# Find the spatial filter SFx
			SFx = spatialFilter(Rx,not_Rx)
			filters += (SFx,)

			# Special case: only two tasks, no need to compute any more mean variances
			if len(tasks) == 2:
				filters += (spatialFilter(not_Rx,Rx),)
				break
		return filters

# covarianceMatrix takes a matrix A and returns the covariance matrix, scaled by the variance
def covarianceMatrix(A):
	Ca = np.dot(A,np.transpose(A))/np.trace(np.dot(A,np.transpose(A)))
	return Ca

# spatialFilter returns the spatial filter SFa for mean covariance matrices Ra and Rb
def spatialFilter(Ra,Rb):
	R = Ra + Rb
	E,U = la.eig(R)

	# CSP requires the eigenvalues E and eigenvector U be sorted in descending order
	ord = np.argsort(E)
	ord = ord[::-1] # argsort gives ascending order, flip to get descending
	E = E[ord]
	U = U[:,ord]

	# Find the whitening transformation matrix
	P = np.dot(np.sqrt(la.inv(np.diag(E))),np.transpose(U))

	# The mean covariance matrices may now be transformed
	Sa = np.dot(P,np.dot(Ra,np.transpose(P)))
	Sb = np.dot(P,np.dot(Rb,np.transpose(P)))

	# Find and sort the generalized eigenvalues and eigenvector
	E1,U1 = la.eig(Sa,Sb)
	ord1 = np.argsort(E1)
	ord1 = ord1[::-1]
	E1 = E1[ord1]
	U1 = U1[:,ord1]

	# The projection matrix (the spatial filter) may now be obtained
	SFa = np.dot(np.transpose(U1),P)
	return SFa.astype(np.float32)
######################################################################
def GetMinSteps(indeces, data):
	minVal = 9999;
	for index in indeces:
		length = data[index].shape[1];
		if(length < minVal):
			minVal = length;
	return minVal
######################################################################
def GetData(indeces, dataIn, truncateValue):
	dataOut = []
	truncate = True;
	if truncateValue == 0:
		truncate = False;
	for idx in indeces:
		currentData = dataIn[idx]
		if truncate:
			dataOut.append(currentData[:,0:truncateValue])
		else:
			dataOut.append(currentData)
	return np.asarray(dataOut)
######################################################################
######################################################################
######################################################################
enableDropout = True;
dropoutPercentage = 0.3;

fileNames = ['HaLTSubjectA1602236StLRHandLegTongue.mat',\
'HaLTSubjectA1603086StLRHandLegTongue.mat',\
'HaLTSubjectA1603106StLRHandLegTongue.mat',\
'HaLTSubjectB1602186StLRHandLegTongue.mat',\
'HaLTSubjectB1602256StLRHandLegTongue.mat',\
'HaLTSubjectB1602296StLRHandLegTongue.mat',\
'HaLTSubjectC1602246StLRHandLegTongue.mat',\
'HaLTSubjectC1603026StLRHandLegTongue.mat',\
'HaLTSubjectE1602196StLRHandLegTongue.mat',\
'HaLTSubjectE1602266StLRHandLegTongue.mat',\
'HaLTSubjectE1603046StLRHandLegTongue.mat',\
'HaLTSubjectF1602026StLRHandLegTongue.mat',\
'HaLTSubjectF1602036StLRHandLegTongue.mat',\
'HaLTSubjectF1602046StLRHandLegTongue.mat',\
'HaLTSubjectG1603016StLRHandLegTongue.mat',\
'HaLTSubjectG1603226StLRHandLegTongue.mat',\
'HaLTSubjectG1604126StLRHandLegTongue.mat',\
'HaLTSubjectH1607206StLRHandLegTongue.mat',\
'HaLTSubjectH1607226StLRHandLegTongue.mat',\
'HaLTSubjectI1606096StLRHandLegTongue.mat',\
'HaLTSubjectI1606286StLRHandLegTongue.mat',\
'HaLTSubjectJ1611216StLRHandLegTongue.mat',\
'HaLTSubjectK1610276StLRHandLegTongue.mat',\
'HaLTSubjectK1611086StLRHandLegTongue.mat',\
'HaLTSubjectL1611166StLRHandLegTongue.mat',\
'HaLTSubjectL1612056StLRHandLegTongue.mat',\
'HaLTSubjectM1611086StLRHandLegTongue.mat',\
'HaLTSubjectM1611176StLRHandLegTongue.mat',\
'HaLTSubjectM1611246StLRHandLegTongue.mat']

numDatasets = len(fileNames);

testAccuracyNoCSP = np.zeros((1,numDatasets))
trainAccuracyNoCSP = np.zeros((1,numDatasets))
testAccuracyWithCSP = np.zeros((1,numDatasets))
trainAccuracyWithCSP = np.zeros((1,numDatasets))

for dataset in range(0, numDatasets):

	fileName = fileNames[dataset];

	file = sio.loadmat('../../../matDown/HaLT_Data/{}'.format(fileName)) #replace with .mat file name
	header=file['__header__']
	version=file['__version__']
	glob=file['__globals__']
	#ans=file['ans']


	#x=file['x']
	o=file['o'][0][0]
	data=o['data']
	data = np.transpose(data)
	data = data[0:21,:];
	print(data)
	nS=o['nS'][0][0]
	#values of structure seem to be 2D numpy arrays, if originally a scalar in Matlab.
	#use [0][0] to get scalar.
	#print("Number of samples: {numSamples}".format(numSamples=nS))
	test=o['id'][0] #id value became a 1D array of size 1 for some reason. use [0] to get value
	#print("Dataset ID: {id}".format(id=test))
	chnames=o['chnames'][:,0] #[:,0] converts from 2D array back to 1D array
	#print("Channel names: {channelNames}".format(channelNames=chnames))
	markers = o['marker']
	## The markers are all still individual arrays of size 1x1, so we convert them to an array with single values
	markersArray = []
	for marker in markers:
		markersArray.append(marker[0])
	markersArray = np.asarray(markersArray)

	#################################
	#################################
	#5F interaction paradigm
	#1-thumb MI, 2-index finger MI, 3-middle finger MI, 4-ring finger MI, 5-pinkie finger MI

	#all paradigms
	#99-initial relaxation period
	#91-inter-session rest break period
	#92-experiment end
	#################################

	## Find the starting indeces where the marker changes
	changeIdxs = np.where(np.transpose(markersArray)[:-1] != np.transpose(markersArray)[1:])[0]
	#print("Number of index changes: {idxChanges}".format(idxChanges=changeIdxs.shape[0]))
	## Split the data so that it has its matching marker
	dataSplit = np.array_split(data, changeIdxs[:-1], axis=1)
	splitCount = 0
	for splitData in dataSplit:
		splitCount += 1
	#print("Number of arrays in data split: {num}".format(num=splitCount))
	## Retrieve the marker values for each of the change indeces (changeIdxs)
	markerTargets = markersArray[changeIdxs];
	#print("Number of marker targets: {numTargets}".format(numTargets=markerTargets.shape[0]))

	## To Apply CSP, we first only get the indeces for MI tasks 1 and 2 (left and right hand, respectively.)
	lhIdx = np.where(markerTargets == 1)[0]
	rhIdx = np.where(markerTargets == 2)[0]
	llIdx = np.where(markerTargets == 4)[0]
	tIdx = np.where(markerTargets == 5)[0]
	rlIdx = np.where(markerTargets == 6)[0]
	
	
	
	
	
	#tCount = tIdx.shape
	#print("Thumb Marker Count: {}\tSize of First: ({},{})".format(tCount, dataSplit[tIdx[0]].shape[0],dataSplit[tIdx[0]].shape[1]))
	#print("Thumb Marker Count: {}\tSize of Second: ({},{})".format(tCount, dataSplit[tIdx[1]].shape[0],dataSplit[tIdx[1]].shape[1]))
	#print("Thumb Marker Count: {}\tSize of Third: ({},{})".format(tCount, dataSplit[tIdx[2]].shape[0],dataSplit[tIdx[2]].shape[1]))


	lhIdxMin = GetMinSteps(lhIdx, dataSplit)
	rhIdxMin = GetMinSteps(rhIdx, dataSplit)
	llIdxMin = GetMinSteps(llIdx, dataSplit)
	tIdxMin = GetMinSteps(tIdx, dataSplit)
	rlIdxMin = GetMinSteps(rlIdx, dataSplit)
	minValues = [lhIdxMin, rhIdxMin, llIdxMin, tIdxMin, rlIdxMin]
	#minValues = [lhIdxMin, rhIdxMin]
	#minValues

	#Truncate the data to the min size
	minValue = np.min(minValues)
	print(minValue)

	lhData = GetData(lhIdx, dataSplit, minValue)
	rhData = GetData(rhIdx, dataSplit, minValue)
	llData = GetData(llIdx, dataSplit, minValue)
	tData = GetData(tIdx, dataSplit, minValue)
	rlData = GetData(rlIdx, dataSplit, minValue)
	#print("Length of tData: {}".format(len(tData)))
	#print("Length of iData: {}".format(len(iData)))
	#print("Length of mData: {}".format(len(mData)))
	#print("Length of rData: {}".format(len(rData)))
	#print("Length of pData: {}".format(len(pData)))

	minLen = np.min([len(lhData), len(rhData), len(llData), len(tData), len(rlData)])
	#minLen = np.min([len(lhData), len(rhData)])
	##Want to make sure they are balanced, so we keep the minLen values
	lhData = lhData[0:minLen]
	rhData = rhData[0:minLen]
	llData = llData[0:minLen]
	tData = tData[0:minLen]
	rlData = rlData[0:minLen]


	# In[68]:


	#Construct the target arrays and merge the data
	lhTargets = np.tile(np.array([1,0,0,0,0]),(minLen,1))
	rhTargets = np.tile(np.array([0,1,0,0,0]),(minLen,1))
	llTargets = np.tile(np.array([0,0,1,0,0]),(minLen,1))
	tTargets = np.tile(np.array([0,0,0,1,0]),(minLen,1))
	rlTargets = np.tile(np.array([0,0,0,0,1]),(minLen,1))
    
	#Print the sizes of each data
	#print("lhData shape: {}".format(lhData.shape))

	markerTargets = np.vstack((lhTargets, rhTargets, llTargets, tTargets, rlTargets))
	#markerTargets = np.vstack((lhTargets, rhTargets))
	Data = np.vstack((lhData, rhData, llData, tData, rlData))
	#Data = np.vstack((lhData, rhData))
    
	#Sanity Check
	#print("Data Shape: {arg1}\tmarkerTargets Shape: {arg2}".format(arg1=Data.shape, arg2=markerTargets.shape))

	## Shuffle the data
	Data, markerTargets = shuffle(Data, markerTargets, random_state=0)

	## Split into train and test sets
	DataTrain, DataTest, markerTargetsTrain, markerTargetsTest = train_test_split(Data, markerTargets, test_size=0.3, random_state=1)
	markerTargetsTrain.shape

	## Reshape the data for time-series processing
	## Syntax np.reshape((numExamples, numTimeSteps, numInputs/numFeatures))
	DataTrainRe = DataTrain.reshape((DataTrain.shape[0], DataTrain.shape[1], DataTrain.shape[2]))
	DataTestRe = DataTest.reshape((DataTest.shape[0], DataTest.shape[1], DataTest.shape[2]))

	## Construct the model
	LSTM_EEG = Sequential()
	LSTM_EEG.add(LSTM((100),batch_input_shape=(None,DataTrainRe.shape[1], DataTrainRe.shape[2]), return_sequences=True))
	if(enableDropout):
		LSTM_EEG.add(Dropout(dropoutPercentage))
	LSTM_EEG.add(LSTM((50), return_sequences=False))
	if(enableDropout):
		LSTM_EEG.add(Dropout(dropoutPercentage))
	LSTM_EEG.add(Dense((5),activation='sigmoid'))

	LSTM_EEG.summary()
	sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
	LSTM_EEG.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])


	# In[79]:


	history = LSTM_EEG.fit(DataTrain, markerTargetsTrain, epochs=30,verbose=2, batch_size=16)


	# In[80]:


	predictionsTest = LSTM_EEG.predict(DataTest)


	# In[81]:


	predictionsTest[predictionsTest>0.5] = 1


	# In[82]:


	predictionsTest[predictionsTest <= 0.5] = 0


	# In[83]:


	comparisonArrayTest = predictionsTest == markerTargetsTest


	# In[85]:


	correctCountTest = 0
	for boolValues in comparisonArrayTest:
		if(boolValues[0] & boolValues[1]):
			correctCountTest += 1
	falseCountTest = DataTest.shape[0] - correctCountTest

	predictionsTrain = LSTM_EEG.predict(DataTrain)
	predictionsTrain[predictionsTrain>0.5] = 1;
	predictionsTrain[predictionsTrain<=0.5] = 0;
	comparisonArrayTrain = predictionsTrain == markerTargetsTrain;

	correctCountTrain = 0
	for boolValues in comparisonArrayTrain:
		if(boolValues[0] & boolValues[1]):
			correctCountTrain += 1
	falseCountTrain = DataTrain.shape[0] - correctCountTrain


	# In[87]:
	trainAcc_noCSP = (correctCountTrain*100/DataTrain.shape[0]);
	testAcc_noCSP = (correctCountTest*100/DataTest.shape[0]);

	#Save these values into the array
	trainAccuracyNoCSP[0,dataset] = trainAcc_noCSP;
	testAccuracyNoCSP[0, dataset] = testAcc_noCSP;

	print("#################################")
	print("#################################")
	print("Training Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTrain, falseCountTrain, trainAcc_noCSP))
	print("#################################")
	print("#################################")
	print("Testing Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTest, falseCountTest, testAcc_noCSP))
	print("#################################")
	print("#################################")


	# In[88]:


	## Applying CSP to 5F data
	filters = CSP(lhData, rhData, llData, tData, rlData)
	#filters = CSP(lhData, rhData)

	# In[90]:


	filtersArray = np.asarray(filters)


	# In[91]:


	filtersArray.shape


	# In[92]:


	lhData_CSP = np.matmul(np.transpose(filtersArray[0]), lhData)
	rhData_CSP = np.matmul(np.transpose(filtersArray[1]), rhData)
	llData_CSP = np.matmul(np.transpose(filtersArray[2]), llData)
	tData_CSP = np.matmul(np.transpose(filtersArray[3]), tData)
	rlData_CSP = np.matmul(np.transpose(filtersArray[4]), rlData)


	# In[93]:


	Data_CSP = np.vstack((lhData_CSP, rhData_CSP, llData_CSP, tData_CSP, rlData_CSP))
	#Data_CSP = np.vstack((lhData_CSP, rhData_CSP))


	# In[94]:


	#Construct the target arrays and merge the data
	lhTargets = np.tile(np.array([1,0,0,0,0]),(minLen,1))
	rhTargets = np.tile(np.array([0,1,0,0,0]),(minLen,1))
	llTargets = np.tile(np.array([0,0,1,0,0]),(minLen,1))
	tTargets = np.tile(np.array([0,0,0,1,0]),(minLen,1))
	rlTargets = np.tile(np.array([0,0,0,0,1]),(minLen,1))

	markerTargets = np.vstack((lhTargets, rhTargets, llTargets, tTargets, rlTargets))
	#markerTargets = np.vstack((lhTargets, rhTargets))

	# In[95]:


	## Shuffle the data
	Data_CSP, markerTargets_CSP = shuffle(Data_CSP, markerTargets, random_state=0)
	## Split into train and test sets
	DataTrain_CSP, DataTest_CSP, markerTargetsTrain_CSP, markerTargetsTest_CSP = train_test_split(Data_CSP, markerTargets_CSP, test_size=0.3, random_state=1)

	##Use the same model to train and test
	history_CSP = LSTM_EEG.fit(DataTrain_CSP, markerTargetsTrain_CSP, epochs=30,verbose=2, batch_size=16)


	# In[96]:


	predictionsTest = LSTM_EEG.predict(DataTest_CSP)


	# In[97]:


	predictionsTest[predictionsTest>0.5] = 1


	# In[98]:


	predictionsTest[predictionsTest <= 0.5] = 0


	# In[99]:


	comparisonArrayTest = predictionsTest == markerTargetsTest


	# In[100]:


	correctCountTest = 0
	for boolValues in comparisonArrayTest:
		if(boolValues[0] & boolValues[1]):
			correctCountTest += 1
	falseCountTest = DataTest_CSP.shape[0] - correctCountTest

	predictionsTrain = LSTM_EEG.predict(DataTrain_CSP)
	predictionsTrain[predictionsTrain>0.5] = 1;
	predictionsTrain[predictionsTrain<=0.5] = 0;
	comparisonArrayTrain = predictionsTrain == markerTargetsTrain;

	correctCountTrain = 0
	for boolValues in comparisonArrayTrain:
		if(boolValues[0] & boolValues[1]):
			correctCountTrain += 1
	falseCountTrain = DataTrain_CSP.shape[0] - correctCountTrain

	#Computing the accuracy
	trainAcc_wCSP = (correctCountTrain*100/DataTrain_CSP.shape[0]);
	testAcc_wCSP = (correctCountTest*100/DataTest_CSP.shape[0]);

	#Save these values into the array
	trainAccuracyWithCSP[0,dataset] = trainAcc_wCSP;
	testAccuracyWithCSP[0, dataset] = testAcc_wCSP;

	print("#################################")
	print("### RESULTS AFTER APPLYING CSP ##")
	print("#################################")
	print("Training Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTrain, falseCountTrain, trainAcc_wCSP))
	print("#################################")
	print("#################################")
	print("Testing Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTest, falseCountTest, testAcc_wCSP))
	print("#################################")
	print("#################################")




[[ -0.    -0.    -0.   ... -36.38 -28.58 -40.94]
 [ -0.    -0.    -0.   ...  -4.69  -5.15  -9.33]
 [ -0.    -0.    -0.   ... -19.24 -20.03 -23.22]
 ...
 [ -0.    -0.    -0.   ...  -7.16  -4.91  -7.68]
 [ -0.    -0.    -0.   ...  -3.36  -2.78  -1.2 ]
 [ -0.    -0.    -0.   ...   3.79   4.74   8.04]]
202
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 21, 100)           121200    
_________________________________________________________________
dropout (Dropout)            (None, 21, 100)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense (Dense)   

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:93: ComplexWarning: Casting complex values to real discards the imaginary part


Epoch 1/30
34/34 - 1s - loss: 0.5904 - accuracy: 0.3720
Epoch 2/30
34/34 - 1s - loss: 0.4207 - accuracy: 0.5121
Epoch 3/30
34/34 - 1s - loss: 0.3945 - accuracy: 0.5271
Epoch 4/30
34/34 - 1s - loss: 0.3742 - accuracy: 0.5439
Epoch 5/30
34/34 - 1s - loss: 0.3566 - accuracy: 0.5664
Epoch 6/30
34/34 - 1s - loss: 0.3406 - accuracy: 0.6187
Epoch 7/30
34/34 - 1s - loss: 0.3341 - accuracy: 0.6093
Epoch 8/30
34/34 - 1s - loss: 0.3330 - accuracy: 0.6243
Epoch 9/30
34/34 - 1s - loss: 0.3265 - accuracy: 0.6280
Epoch 10/30
34/34 - 1s - loss: 0.3206 - accuracy: 0.6168
Epoch 11/30
34/34 - 1s - loss: 0.3114 - accuracy: 0.6542
Epoch 12/30
34/34 - 1s - loss: 0.3094 - accuracy: 0.6617
Epoch 13/30
34/34 - 1s - loss: 0.2848 - accuracy: 0.7103
Epoch 14/30
34/34 - 1s - loss: 0.2846 - accuracy: 0.6879
Epoch 15/30
34/34 - 1s - loss: 0.2900 - accuracy: 0.6991
Epoch 16/30
34/34 - 1s - loss: 0.2781 - accuracy: 0.7346
Epoch 17/30
34/34 - 1s - loss: 0.2711 - accuracy: 0.7234
Epoch 18/30
34/34 - 1s - loss: 0.2530 - 

34/34 - 1s - loss: 0.5540 - accuracy: 0.2000
Epoch 2/30
34/34 - 1s - loss: 0.4742 - accuracy: 0.3944
Epoch 3/30
34/34 - 1s - loss: 0.4308 - accuracy: 0.5458
Epoch 4/30
34/34 - 1s - loss: 0.3758 - accuracy: 0.6430
Epoch 5/30
34/34 - 1s - loss: 0.3133 - accuracy: 0.7103
Epoch 6/30
34/34 - 1s - loss: 0.2777 - accuracy: 0.7607
Epoch 7/30
34/34 - 1s - loss: 0.2624 - accuracy: 0.7477
Epoch 8/30
34/34 - 1s - loss: 0.2328 - accuracy: 0.7944
Epoch 9/30
34/34 - 1s - loss: 0.2485 - accuracy: 0.7589
Epoch 10/30
34/34 - 1s - loss: 0.2175 - accuracy: 0.8206
Epoch 11/30
34/34 - 1s - loss: 0.1851 - accuracy: 0.8411
Epoch 12/30
34/34 - 1s - loss: 0.1603 - accuracy: 0.8710
Epoch 13/30
34/34 - 1s - loss: 0.1441 - accuracy: 0.8785
Epoch 14/30
34/34 - 1s - loss: 0.1720 - accuracy: 0.8393
Epoch 15/30
34/34 - 1s - loss: 0.1734 - accuracy: 0.8355
Epoch 16/30
34/34 - 1s - loss: 0.1830 - accuracy: 0.8411
Epoch 17/30
34/34 - 1s - loss: 0.1511 - accuracy: 0.8766
Epoch 18/30
34/34 - 1s - loss: 0.1229 - accuracy: 0

Epoch 11/30
34/34 - 1s - loss: 0.2335 - accuracy: 0.7650
Epoch 12/30
34/34 - 1s - loss: 0.2169 - accuracy: 0.8008
Epoch 13/30
34/34 - 1s - loss: 0.1978 - accuracy: 0.8158
Epoch 14/30
34/34 - 1s - loss: 0.2082 - accuracy: 0.7989
Epoch 15/30
34/34 - 1s - loss: 0.2206 - accuracy: 0.7989
Epoch 16/30
34/34 - 1s - loss: 0.2407 - accuracy: 0.7556
Epoch 17/30
34/34 - 1s - loss: 0.2042 - accuracy: 0.8177
Epoch 18/30
34/34 - 1s - loss: 0.1796 - accuracy: 0.8571
Epoch 19/30
34/34 - 1s - loss: 0.1630 - accuracy: 0.8515
Epoch 20/30
34/34 - 1s - loss: 0.1655 - accuracy: 0.8590
Epoch 21/30
34/34 - 1s - loss: 0.1650 - accuracy: 0.8684
Epoch 22/30
34/34 - 1s - loss: 0.1585 - accuracy: 0.8759
Epoch 23/30
34/34 - 1s - loss: 0.1489 - accuracy: 0.8647
Epoch 24/30
34/34 - 1s - loss: 0.1450 - accuracy: 0.8741
Epoch 25/30
34/34 - 1s - loss: 0.1322 - accuracy: 0.8778
Epoch 26/30
34/34 - 1s - loss: 0.1456 - accuracy: 0.8759
Epoch 27/30
34/34 - 1s - loss: 0.1596 - accuracy: 0.8515
Epoch 28/30
34/34 - 1s - loss: 

...

34/34 - 1s - loss: 0.0937 - accuracy: 0.9364
Epoch 13/30
34/34 - 1s - loss: 0.0751 - accuracy: 0.9439
Epoch 14/30
34/34 - 1s - loss: 0.0683 - accuracy: 0.9477
Epoch 15/30
34/34 - 1s - loss: 0.0680 - accuracy: 0.9477
Epoch 16/30
34/34 - 1s - loss: 0.0624 - accuracy: 0.9551
Epoch 17/30
34/34 - 1s - loss: 0.0496 - accuracy: 0.9701
Epoch 18/30
34/34 - 1s - loss: 0.0433 - accuracy: 0.9757
Epoch 19/30
34/34 - 1s - loss: 0.0572 - accuracy: 0.9533
Epoch 20/30
34/34 - 1s - loss: 0.0580 - accuracy: 0.9626
Epoch 21/30
34/34 - 1s - loss: 0.0504 - accuracy: 0.9589
Epoch 22/30
34/34 - 1s - loss: 0.0511 - accuracy: 0.9626
Epoch 23/30
34/34 - 1s - loss: 0.0597 - accuracy: 0.9495
Epoch 24/30
34/34 - 1s - loss: 0.0408 - accuracy: 0.9720
Epoch 25/30
34/34 - 1s - loss: 0.0399 - accuracy: 0.9738
Epoch 26/30
34/34 - 1s - loss: 0.0340 - accuracy: 0.9776
Epoch 27/30
34/34 - 1s - loss: 0.0330 - accuracy: 0.9776
Epoch 28/30
34/34 - 0s - loss: 0.0465 - accuracy: 0.9701
Epoch 29/30
34/34 - 0s - loss: 0.0376 - acc

34/34 - 0s - loss: 0.5362 - accuracy: 0.2857
Epoch 2/30
34/34 - 0s - loss: 0.4592 - accuracy: 0.4135
Epoch 3/30
34/34 - 0s - loss: 0.4119 - accuracy: 0.5056
Epoch 4/30
34/34 - 0s - loss: 0.3649 - accuracy: 0.6053
Epoch 5/30
34/34 - 0s - loss: 0.3083 - accuracy: 0.7237
Epoch 6/30
34/34 - 0s - loss: 0.2554 - accuracy: 0.7782
Epoch 7/30
34/34 - 0s - loss: 0.2710 - accuracy: 0.7293
Epoch 8/30
34/34 - 0s - loss: 0.2470 - accuracy: 0.7801
Epoch 9/30
34/34 - 0s - loss: 0.1805 - accuracy: 0.8853
Epoch 10/30
34/34 - 1s - loss: 0.1553 - accuracy: 0.8609
Epoch 11/30
34/34 - 1s - loss: 0.1614 - accuracy: 0.8703
Epoch 12/30
34/34 - 1s - loss: 0.2276 - accuracy: 0.7932
Epoch 13/30
34/34 - 1s - loss: 0.1411 - accuracy: 0.8947
Epoch 14/30
34/34 - 1s - loss: 0.1378 - accuracy: 0.8910
Epoch 15/30
34/34 - 1s - loss: 0.0995 - accuracy: 0.9267
Epoch 16/30
34/34 - 1s - loss: 0.1257 - accuracy: 0.9023
Epoch 17/30
34/34 - 1s - loss: 0.1172 - accuracy: 0.9173
Epoch 18/30
34/34 - 1s - loss: 0.0993 - accuracy: 0

34/34 - 1s - loss: 0.1491 - accuracy: 0.8766
Epoch 13/30
34/34 - 1s - loss: 0.1467 - accuracy: 0.8748
Epoch 14/30
34/34 - 1s - loss: 0.1355 - accuracy: 0.8822
Epoch 15/30
34/34 - 1s - loss: 0.1148 - accuracy: 0.8991
Epoch 16/30
34/34 - 1s - loss: 0.1329 - accuracy: 0.8841
Epoch 17/30
34/34 - 1s - loss: 0.1358 - accuracy: 0.8860
Epoch 18/30
34/34 - 1s - loss: 0.1282 - accuracy: 0.8841
Epoch 19/30
34/34 - 1s - loss: 0.1100 - accuracy: 0.9065
Epoch 20/30
34/34 - 1s - loss: 0.0997 - accuracy: 0.9234
Epoch 21/30
34/34 - 1s - loss: 0.0974 - accuracy: 0.9215
Epoch 22/30
34/34 - 1s - loss: 0.1061 - accuracy: 0.9121
Epoch 23/30
34/34 - 1s - loss: 0.0962 - accuracy: 0.9159
Epoch 24/30
34/34 - 1s - loss: 0.0894 - accuracy: 0.9383
Epoch 25/30
34/34 - 1s - loss: 0.0737 - accuracy: 0.9383
Epoch 26/30
34/34 - 1s - loss: 0.0672 - accuracy: 0.9514
Epoch 27/30
34/34 - 1s - loss: 0.0971 - accuracy: 0.9290
Epoch 28/30
34/34 - 1s - loss: 0.0664 - accuracy: 0.9607
Epoch 29/30
34/34 - 1s - loss: 0.0564 - acc

34/34 - 1s - loss: 0.5450 - accuracy: 0.2131
Epoch 2/30
34/34 - 0s - loss: 0.4663 - accuracy: 0.4467
Epoch 3/30
34/34 - 0s - loss: 0.4136 - accuracy: 0.5963
Epoch 4/30
34/34 - 0s - loss: 0.3458 - accuracy: 0.6654
Epoch 5/30
34/34 - 0s - loss: 0.2976 - accuracy: 0.7271
Epoch 6/30
34/34 - 0s - loss: 0.2583 - accuracy: 0.7701
Epoch 7/30
34/34 - 0s - loss: 0.2261 - accuracy: 0.8206
Epoch 8/30
34/34 - 0s - loss: 0.2218 - accuracy: 0.8019
Epoch 9/30
34/34 - 1s - loss: 0.2198 - accuracy: 0.7888
Epoch 10/30
34/34 - 1s - loss: 0.1918 - accuracy: 0.8224
Epoch 11/30
34/34 - 1s - loss: 0.1782 - accuracy: 0.8598
Epoch 12/30
34/34 - 1s - loss: 0.1398 - accuracy: 0.8804
Epoch 13/30
34/34 - 1s - loss: 0.1330 - accuracy: 0.9028
Epoch 14/30
34/34 - 1s - loss: 0.2019 - accuracy: 0.8168
Epoch 15/30
34/34 - 1s - loss: 0.1615 - accuracy: 0.8505
Epoch 16/30
34/34 - 1s - loss: 0.1736 - accuracy: 0.8561
Epoch 17/30
34/34 - 1s - loss: 0.1337 - accuracy: 0.8841
Epoch 18/30
34/34 - 1s - loss: 0.1153 - accuracy: 0

Epoch 11/30
34/34 - 1s - loss: 0.1874 - accuracy: 0.8243
Epoch 12/30
34/34 - 1s - loss: 0.2021 - accuracy: 0.8150
Epoch 13/30
34/34 - 1s - loss: 0.1694 - accuracy: 0.8654
Epoch 14/30
34/34 - 1s - loss: 0.1760 - accuracy: 0.8411
Epoch 15/30
34/34 - 1s - loss: 0.1770 - accuracy: 0.8449
Epoch 16/30
34/34 - 1s - loss: 0.1583 - accuracy: 0.8636
Epoch 17/30
34/34 - 1s - loss: 0.1681 - accuracy: 0.8467
Epoch 18/30
34/34 - 1s - loss: 0.1557 - accuracy: 0.8505
Epoch 19/30
34/34 - 1s - loss: 0.1241 - accuracy: 0.8972
Epoch 20/30
34/34 - 1s - loss: 0.1365 - accuracy: 0.8953
Epoch 21/30
34/34 - 1s - loss: 0.1354 - accuracy: 0.8897
Epoch 22/30
34/34 - 1s - loss: 0.1173 - accuracy: 0.9103
Epoch 23/30
34/34 - 1s - loss: 0.1435 - accuracy: 0.8766
Epoch 24/30
34/34 - 1s - loss: 0.1295 - accuracy: 0.8879
Epoch 25/30
34/34 - 1s - loss: 0.1152 - accuracy: 0.9084
Epoch 26/30
34/34 - 1s - loss: 0.1151 - accuracy: 0.8916
Epoch 27/30
34/34 - 1s - loss: 0.1051 - accuracy: 0.9308
Epoch 28/30
34/34 - 1s - loss: 

In [2]:
fileNames

['HaLTSubjectA1602236StLRHandLegTongue.mat',
 'HaLTSubjectA1603086StLRHandLegTongue.mat',
 'HaLTSubjectA1603106StLRHandLegTongue.mat',
 'HaLTSubjectB1602186StLRHandLegTongue.mat',
 'HaLTSubjectB1602256StLRHandLegTongue.mat',
 'HaLTSubjectB1602296StLRHandLegTongue.mat',
 'HaLTSubjectC1602246StLRHandLegTongue.mat',
 'HaLTSubjectC1603026StLRHandLegTongue.mat',
 'HaLTSubjectE1602196StLRHandLegTongue.mat',
 'HaLTSubjectE1602266StLRHandLegTongue.mat',
 'HaLTSubjectE1603046StLRHandLegTongue.mat',
 'HaLTSubjectF1602026StLRHandLegTongue.mat',
 'HaLTSubjectF1602036StLRHandLegTongue.mat',
 'HaLTSubjectF1602046StLRHandLegTongue.mat',
 'HaLTSubjectG1603016StLRHandLegTongue.mat',
 'HaLTSubjectG1603226StLRHandLegTongue.mat',
 'HaLTSubjectG1604126StLRHandLegTongue.mat',
 'HaLTSubjectH1607206StLRHandLegTongue.mat',
 'HaLTSubjectH1607226StLRHandLegTongue.mat',
 'HaLTSubjectI1606096StLRHandLegTongue.mat',
 'HaLTSubjectI1606286StLRHandLegTongue.mat',
 'HaLTSubjectJ1611216StLRHandLegTongue.mat',
 'HaLTSubj

In [3]:
testAccuracyNoCSP

array([[57.82608696, 56.52173913, 63.91304348, 50.        , 48.80382775,
        48.69565217, 66.95652174, 55.65217391, 50.43478261, 52.67857143,
        54.38596491, 59.13043478, 57.82608696, 44.34782609, 67.39130435,
        56.57894737, 66.08695652, 46.69603524, 50.23923445, 55.81395349,
        48.24561404, 80.86956522, 62.60869565, 53.07017544, 59.13043478,
        67.10526316, 63.47826087, 57.82608696, 56.14035088]])

In [4]:
testAccuracyWithCSP

array([[69.56521739, 72.46376812, 75.2173913 , 79.38596491, 67.46411483,
        56.52173913, 78.69565217, 69.13043478, 74.7826087 , 69.19642857,
        67.98245614, 74.7826087 , 76.52173913, 66.52173913, 93.04347826,
        86.40350877, 87.82608696, 55.50660793, 67.94258373, 70.69767442,
        57.45614035, 96.08695652, 78.69565217, 70.1754386 , 84.34782609,
        88.59649123, 94.34782609, 79.56521739, 67.10526316]])

In [5]:
trainAccuracyNoCSP

array([[ 97.75700935,  98.13664596,  96.82242991,  98.12030075,
         98.14814815,  99.62616822,  99.25233645,  90.28037383,
         99.25233645,  99.42418426,  97.93233083,  97.94392523,
         99.81308411,  97.75700935,  97.94392523,  97.93233083,
         99.25233645,  93.18181818,  88.06584362,  98.        ,
         79.5112782 ,  99.81308411,  96.82242991,  99.2481203 ,
         99.06542056,  99.81203008,  98.87850467, 100.        ,
         99.62406015]])

In [6]:
trainAccuracyWithCSP

array([[ 93.27102804,  94.82401656,  94.39252336,  95.4887218 ,
         95.67901235,  83.36448598,  95.70093458,  82.42990654,
         92.71028037,  90.21113244,  87.40601504,  92.71028037,
         98.69158879,  96.63551402,  98.87850467,  98.4962406 ,
         96.63551402,  73.67424242,  73.04526749,  89.8       ,
         72.18045113,  99.62616822,  97.19626168,  91.35338346,
         99.06542056,  99.43609023, 100.        ,  97.57009346,
         92.66917293]])

In [7]:
combinedArray = np.transpose(np.asarray(np.vstack((trainAccuracyNoCSP,testAccuracyNoCSP,trainAccuracyWithCSP, testAccuracyWithCSP))))

In [8]:
combinedArray.shape

(29, 4)

In [10]:
np.savetxt('HaLTperSubject_Dropout.csv', combinedArray, delimiter=',')